In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random
from shutil import copyfile
import torch.nn.functional as F
from torch.autograd import Variable

In [0]:
def train_test_split(targetfolder,ratio):
    classes = os.listdir(targetfolder)
#  Example: train_test_split("Plots_v2_clean","Plots_v2_clean_split",{6:450,7:200,10:650,12:650})
    newfolder = targetfolder+'_split'
    os.mkdir(newfolder)
    for j in ["train","val"]:
        os.mkdir(newfolder+"/"+j)
        for i in classes:    
            os.mkdir(newfolder+"/"+j+"/"+str(i))
    for i in classes:
        path = targetfolder+"/"+str(i)+"/"
        imglist = os.listdir(path)
        train_list = random.sample(imglist,int(len(imglist)*ratio))
        val_list = list(set(imglist) - set(train_list)) 
        for j in train_list:
            oldpath = path+j
            newpath = newfolder+"/"+"train"+"/"+str(i)+"/"+j
            copyfile(oldpath,newpath)
        for j in val_list:
            oldpath = path+j
            newpath = newfolder+"/"+"val"+"/"+str(i)+"/"+j
            copyfile(oldpath,newpath)
train_test_split('/content/drive/My Drive/DS100 final project/20_categories_training',0.8)

In [0]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(10),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/drive/My Drive/DS100 final project/20_categories_training_split'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(64, 192, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 224, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(224, 224, kernel_size=3, padding=1)
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(224 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 20),
        )
#https://github.com/pytorch/vision/blob/master/torchvision/models/alexnet.py

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(F.relu(self.conv5(x)))
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
net = Net()

train_loss_list = []
val_loss_list = []

In [0]:


def validation_accuracy():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloaders['val']:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))
    return correct / total
            

def train(net,epoch):
    net.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    best_acc = 0.59
    for epoch in range(epoch):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(dataloaders['train'], 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            l1_regularization = 0
            lambda_ = 0.00001
            for param in net.parameters():
                l1_regularization += torch.norm(param, 1)
            loss = loss + lambda_*l1_regularization 
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

        print('[%d] loss: %.3f' %
                (epoch + 1, running_loss / len(dataloaders['train'])))
        val_acc = validation_accuracy()
        if val_acc>best_acc:
            path = '/content/drive/My Drive/DS100 final project/CNN.pth'
            best_acc = val_acc
            torch.save(net.state_dict(), PATH)
            print('saved model')
        running_loss = 0.0

    print('Finished Training')


In [7]:
train(net,1)

[1] loss: 4.339
Accuracy of the network on the test images: 57 %
Finished Training


In [0]:
PATH = '/content/drive/My Drive/DS100 final project/CNN.pth'
torch.save(best_model.state_dict(), PATH)

In [4]:
PATH = '/content/drive/My Drive/DS100 final project/CNN.pth'
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>